# Daily Update & Tweet Notebook

- Running for the first time in the day may take 5-7 minutes
- Updating should take more like 3 minutes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import random
import shutil
import plotly
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests     
from scipy.stats import norm
import winsound
import tweepy
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import WebDriverException

## 1) Download Missing Data

- Use the .py files previously created to download the missing data.
- Use apply & lambda to apply the function to each new row of the dataframe.

In [2]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')

%run player_box_scores.py

 last game date in df: 2023-01-22
data/player/box_scores/boxscores-traditional__Season_2022-23_Month_4_Regular.csv Completed Successfully! 1 / 2 Complete!
data/player/box_scores/boxscores-advanced__Season_2022-23_Month_4_Regular.csv Completed Successfully! 2 / 2 Complete!
Data is Now Updated


In [3]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')
%run team_box_score_update.py

yesterday is not later than latest date, so do not run the code


### 2) Get Yesterday's Games and Odds

In [4]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')

%run bigdataball_scrape.py

 Last date uploaded: 01/24/2023
Yesterday was: 01/24/2023
Already up to date


### 3) Update Features of Yesterday's Games

This takes 2-5 minutes

Inputs: 
- old df final: data/team/aggregates/daily_updates/Updated_All_Things_DATE.csv

In [5]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')
%run update_all_features_v2.py


most recent file is:  Updated_All_Things_2023-01-25.csv
(23290, 919)
54
 You have 0 games to add to your dataset
file already exists


### 4) Append new to 'old df'

Old_DF is just the dataframe that contains all the data from the previous day.


In [6]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
today

'2023-01-25'

In [7]:
new_df = pd.read_csv('data/team/aggregates/daily_updates/'+str(today)+'.csv')
new_df.head()

,Unnamed: 0,trad_unnamed: 0,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,...,Team_Abrev,Nba_com_team_Abbrev,Opp_Abbrev,Nba_com_team_2_abbrev,Date_underscore,HomeTeam,AwayTeam,Matchup_GameDate_y,date,opponent
0,29917,8,PHX,PHX vs. CHA,2023-01-24,W,48.0,128.0,47.0,94.0,...,Pho,PHX,Cha,CHA,01_24_2023,PHX,CHA,CHA @ PHX_01_24_2023,2023-01-24,CHA
1,29920,11,BOS,BOS @ MIA,2023-01-24,L,48.0,95.0,32.0,74.0,...,Bos,BOS,Mia,MIA,01_24_2023,MIA,BOS,BOS @ MIA_01_24_2023,2023-01-24,MIA
2,29916,7,DAL,DAL vs. WAS,2023-01-24,L,48.0,126.0,47.0,89.0,...,Dal,DAL,Was,WAS,01_24_2023,DAL,WAS,WAS @ DAL_01_24_2023,2023-01-24,WAS
3,29910,1,DEN,DEN @ NOP,2023-01-24,W,48.0,99.0,39.0,80.0,...,Den,DEN,Nor,NOP,01_24_2023,NOP,DEN,DEN @ NOP_01_24_2023,2023-01-24,NOP
4,29911,2,CHA,CHA @ PHX,2023-01-24,L,48.0,97.0,39.0,96.0,...,Cha,CHA,Pho,PHX,01_24_2023,PHX,CHA,CHA @ PHX_01_24_2023,2023-01-24,PHX


In [8]:
# drop any games that are from today
new_df = new_df[new_df['trad_gamedate'] != today]

### 4) Get Today's Games and Odds

In [9]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')


In [10]:
# Use draftkings scrape
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'
driver = webdriver.Chrome()
# minimize window

driver.get(url)
# wait for page to load
time.sleep(3)
driver.minimize_window()

# find table (class sportsbook-table)
table = driver.find_element_by_class_name('sportsbook-table')
# get all html in table
table_html = table.get_attribute('innerHTML')
# convert to soup
soup = BeautifulSoup(table_html, 'html.parser')
# find all rows
rows = soup.find_all('tr')
# find all headers
headers = table.find_element_by_class_name('sportsbook-table__head')
headers = headers.find_elements_by_tag_name('th')
headers = [header.text for header in headers]
headers

# find all rows
rows = table.find_elements_by_tag_name('tr')
# get all html in each row
rows_html = [row.get_attribute('innerHTML') for row in rows]
# convert to soup
rows_soup = [BeautifulSoup(row, 'html.parser') for row in rows_html]
# find all cells in each row
rows_cells = [row.find_all('td') for row in rows_soup]
# drop first row (headers)
rows_cells = rows_cells[1:]
# get text from each cell
rows_cells_text = [[cell.text for cell in row] for row in rows_cells]
names = table.find_elements_by_class_name('event-cell__name')
names = [name.text for name in names]

# add names to rows_cells_text
for i in range(len(rows_cells_text)):
    rows_cells_text[i].insert(0, names[i])

# convert to dataframe
rows_cells_text
df = pd.DataFrame(rows_cells_text, columns=headers)
df['spread_odds'] = df['SPREAD'].str[-4:]
df['SPREAD'] = df['SPREAD'].str[:-4]
df['total_odds'] = df['O/U'].str[-4:]
df['TOTAL'] = df['O/U'].str[:-4]

df.to_csv('data/team/aggregates/daily_updates/draftkings' + str(today) + '.csv', index=False)
df

# close driver
driver.close()


### Create DF with today's games and odds

In [11]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')
%run todays_games_df_Creation.py


old df final shape: (22824, 834)
creating league running averages
getting rest days
Calculating NBA Averages
saved data/team/aggregates/daily_updates/today_df_features__2023-01-25.csv
Job Done!


In [12]:
today_df = pd.read_csv('data/team/aggregates/daily_updates/today_df_features__' + str(today) + '.csv')
today_df.head()

,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,Estimate_Points_Difference_40gm,Tm1_Points_Estimate_20gm,Tm2_Points_Estimate_20gm,Estimate_Points_Difference_20gm,Tm1_Points_Estimate_10gm,Tm2_Points_Estimate_10gm,Estimate_Points_Difference_10gm,Tm1_Points_Estimate_Weighted,Tm2_Points_Estimate_Weighted,Estimate_Points_Difference_Weighted
0,0,IND Pacers,5.5,O 234−110,+165,−115,−110,234.0,IND,ORL,...,1.702227,113.230637,119.163042,-5.932405,115.232270,121.409110,-6.176840,113.196049,116.252836,-3.056787
1,1,ORL Magic,-5.5,U 234−110,−195,−105,−110,234.0,ORL,IND,...,-1.702227,119.163042,113.230637,5.932405,121.409110,115.232270,6.176840,116.252836,113.196049,3.056787
2,2,BKN Nets,6.0,O 224.5−110,+190,−105,−110,224.5,BKN,PHI,...,-1.558217,120.127344,116.020505,4.106839,126.551076,124.565416,1.985660,120.409862,118.949371,1.460492
3,3,PHI 76ers,-6.0,U 224.5−110,−225,−115,−110,224.5,PHI,BKN,...,1.558217,116.020505,120.127344,-4.106839,124.565416,126.551076,-1.985660,118.949371,120.409862,-1.460492
4,4,ATL Hawks,-1.5,O 238−110,−125,−110,−110,238.0,ATL,OKC,...,0.545577,112.539423,112.834515,-0.295092,122.906163,117.349358,5.556806,116.622827,114.203091,2.419737


### Get Roto Defense by Position

In [13]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')

%run Roto_defense_by_position_scrape.py

### Get Expected Minutes (Rotowire)

In [14]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')

%run Rotowire_expected_minutes.py

Job Done


In [15]:
# load in rotowire data
today = datetime.datetime.today().strftime('%Y-%m-%d')

expected_minutes = pd.read_csv('data/team/aggregates/daily_updates/player_minutes_projection_{}.csv'.format(today))
expected_minutes.head()

,name,minutes,OUT,GTD,team
0,Buddy Hield,30.0,0,0,Pacers
1,T.J. McConnell,28.0,0,0,Pacers
2,Andrew Nembhard,28.0,0,1,Pacers
3,Myles Turner,28.0,0,0,Pacers
4,Aaron Nesmith,26.0,0,1,Pacers


In [16]:
# get starters
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')
%run Rotowire_players_playing_today.py


### Player Metric Additions

In [17]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')

season_averages = pd.read_csv('data/player/aggregates_of_aggregates/trad_season_averages_imputed.csv')
season_averages.drop_duplicates(inplace=True)
season_averages.head()

,Unnamed: 0.1,trad_player,trad_team,trad_season,Unnamed: 0,trad_unnamed: 0,trad_min,trad_pts,trad_fgm,trad_fga,...,Shotclock_FG,Shotclock_Attempts,PM_Pressure_Scoring,Drive_FG,Drive_Attempts,PM_Driving_Offense,PM_Passing,CS_3P%,CS_3PA,PM_Catch_Shoot
0,0,AJ Green,MIL,2022,257966.0,1695.0,6.0,3.0,1.0,2.0,...,NaN,NaN,0.459336,0.994444,0.040278,0.755903,0.300000,0.983333,0.200000,0.787500
1,1,AJ Griffin,ATL,2022,257318.0,1738.0,21.0,11.0,4.0,9.0,...,NaN,NaN,0.415369,0.576720,0.677249,0.601852,0.412698,0.624339,0.751323,0.656085
2,2,AJ Hammons,DAL,2016,114129.0,2013.0,7.0,2.0,1.0,2.0,...,NaN,NaN,0.488249,0.068245,0.077994,0.070682,0.398329,0.983287,0.701950,0.912953
3,3,AJ Hammons,DAL,2016,114129.0,2013.0,7.0,2.0,1.0,2.0,...,NaN,NaN,0.398766,0.068245,0.077994,0.070682,0.398329,0.983287,0.701950,0.912953
4,4,AJ Hammons,DAL,2016,114129.0,2013.0,7.0,2.0,1.0,2.0,...,NaN,NaN,0.502086,0.068245,0.077994,0.070682,0.398329,0.983287,0.701950,0.912953


In [18]:
season_averages = season_averages[season_averages['trad_season'] == 2021]
# add player metrics from season averages to expected minutes
expected_minutes = expected_minutes.merge(season_averages, left_on= 'name', right_on='trad_player', how='left')
expected_minutes = expected_minutes.drop_duplicates(subset= ['name'])
expected_minutes.head()

,name,minutes,OUT,GTD,team_x,Unnamed: 0.1,trad_player,trad_team,trad_season,Unnamed: 0,...,Shotclock_FG,Shotclock_Attempts,PM_Pressure_Scoring,Drive_FG,Drive_Attempts,PM_Driving_Offense,PM_Passing,CS_3P%,CS_3PA,PM_Catch_Shoot
0,Buddy Hield,30.0,0,0,Pacers,12417.0,Buddy Hield,IND,2021.0,247051.0,...,0.460957,0.778338,0.540302,0.390121,0.671371,0.460433,0.449597,0.692540,0.960685,0.759577
180,T.J. McConnell,28.0,0,0,Pacers,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,Andrew Nembhard,28.0,0,1,Pacers,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,Myles Turner,28.0,0,0,Pacers,78555.0,Myles Turner,IND,2021.0,232921.0,...,0.716321,0.326425,0.618847,0.619221,0.894161,0.687956,0.336983,0.619221,0.950122,0.701946
200,Aaron Nesmith,26.0,0,1,Pacers,510.0,Aaron Nesmith,BOS,2021.0,237671.0,...,0.810954,0.234982,0.666961,0.840909,0.410714,0.733360,0.125000,0.266234,0.441558,0.310065


In [19]:


expected_minutes['PM_TOV_Ratio_flipped'] = 1 - expected_minutes['PM_TOV_Rato']
expected_minutes['PM_Iso_Defense_flipped'] = 1 - expected_minutes['PM_Iso_Defense']
expected_minutes['PM_defensive_FG_flipped'] = 1 - expected_minutes['PM_defensive_FG']

expected_minutes['PM_Aggregate_Offensive_Rating'] = ((expected_minutes['PM_True_Shooting'] * .1) +
                                        (expected_minutes['PM_Open_3'] * .1) +
                                        (expected_minutes['PM_Ball_Handler'] * .1) +
                                        (expected_minutes['PM_Iso'] * .15) +
                                        (expected_minutes['PM_Shot_Creation'] * .1) +
                                        (expected_minutes['PM_Pressure_Scoring'] * .05) +
                                        (expected_minutes['PM_Driving_Offense'] * .05) +
                                        (expected_minutes['PM_Passing'] * .1) +
                                        (expected_minutes['PM_Catch_Shoot'] * .15) +
                                        (expected_minutes['PM_TOV_Ratio_flipped'] * .1))


expected_minutes['PM_Aggregate_Defensive_Rating'] = ( (((expected_minutes['PM_matchup_difficulty'] * .15) /100) + 
                                        (expected_minutes['PM_Iso_Defense_flipped'] * .1) + 
                                        (expected_minutes['PM_loose_balls_recovered'] * .1) +
                                        (expected_minutes['PM_speed_score'] * .1) +
                                        (expected_minutes['PM_Adv_Rebounding'] * .0375) +
                                        (expected_minutes['PM_Trad_Rebounding'] * .0375) +
                                        (expected_minutes['PM_offensive_boxouts'] * .0375) +
                                        (expected_minutes['PM_defensive_boxouts'] * .0375) +
                                        (expected_minutes['PM_coverage_area'] * .2) +
                                        (expected_minutes['PM_defensive_FG_flipped'] * .2)))

expected_minutes['PM_Player_Metric'] = expected_minutes['PM_Aggregate_Offensive_Rating'] + expected_minutes['PM_Aggregate_Defensive_Rating']

In [20]:
keep_cols = ['name', 'minutes', 'OUT', 'GTD', 'team_x', 'PM_Aggregate_Offensive_Rating', 
                'PM_Aggregate_Defensive_Rating', 'PM_Player_Metric']

expected_minutes2 = expected_minutes[keep_cols]
expected_minutes2.head(2)

,name,minutes,OUT,GTD,team_x,PM_Aggregate_Offensive_Rating,PM_Aggregate_Defensive_Rating,PM_Player_Metric
0,Buddy Hield,30.0,0,0,Pacers,0.630508,0.489893,1.120402
180,T.J. McConnell,28.0,0,0,Pacers,NaN,NaN,NaN


In [21]:
# TODO: Impute the rookies scores
# Likely, will have to encorporate size, pick number, position, and stats so far this year (wait 20 games?)

In [22]:
# team dictionary (names to abbreviations)
team_dict = {'Hawks':'ATL',
                'Nets':'BKN',
                'Celtics':'BOS',
                'Hornets':'CHA',
                'Bulls':'CHI',
                'Cavaliers':'CLE',
                'Mavericks':'DAL',
                'Nuggets':'DEN',
                'Pistons':'DET',
                'Warriors':'GSW',
                'Rockets':'HOU',
                'Pacers':'IND',
                'Clippers':'LAC',
                'Lakers':'LAL',
                'Grizzlies':'MEM',
                'Heat':'MIA',
                'Bucks':'MIL',
                'Timberwolves':'MIN',
                'Pelicans':'NOP',
                'Knicks':'NYK',
                'Thunder':'OKC',
                'Magic':'ORL',
                '76ers':'PHI',
                'Suns':'PHX',
                'Trail Blazers':'POR',
                'Kings':'SAC',
                'Spurs':'SAS',
                'Raptors':'TOR',
                'Jazz':'UTA',
                'Wizards':'WAS'}

In [23]:
expected_minutes2['trad_team'] = expected_minutes2['team_x'].map(team_dict)
expected_minutes2.head()

,name,minutes,OUT,GTD,team_x,PM_Aggregate_Offensive_Rating,PM_Aggregate_Defensive_Rating,PM_Player_Metric,trad_team
0,Buddy Hield,30.0,0,0,Pacers,0.630508,0.489893,1.120402,IND
180,T.J. McConnell,28.0,0,0,Pacers,NaN,NaN,NaN,IND
181,Andrew Nembhard,28.0,0,1,Pacers,NaN,NaN,NaN,IND
182,Myles Turner,28.0,0,0,Pacers,0.533866,0.526067,1.059932,IND
200,Aaron Nesmith,26.0,0,1,Pacers,0.315441,0.562098,0.877539,IND


## All Together Now

#### Add Team Player Metrics

In [24]:
today_df.head()

,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,Estimate_Points_Difference_40gm,Tm1_Points_Estimate_20gm,Tm2_Points_Estimate_20gm,Estimate_Points_Difference_20gm,Tm1_Points_Estimate_10gm,Tm2_Points_Estimate_10gm,Estimate_Points_Difference_10gm,Tm1_Points_Estimate_Weighted,Tm2_Points_Estimate_Weighted,Estimate_Points_Difference_Weighted
0,0,IND Pacers,5.5,O 234−110,+165,−115,−110,234.0,IND,ORL,...,1.702227,113.230637,119.163042,-5.932405,115.232270,121.409110,-6.176840,113.196049,116.252836,-3.056787
1,1,ORL Magic,-5.5,U 234−110,−195,−105,−110,234.0,ORL,IND,...,-1.702227,119.163042,113.230637,5.932405,121.409110,115.232270,6.176840,116.252836,113.196049,3.056787
2,2,BKN Nets,6.0,O 224.5−110,+190,−105,−110,224.5,BKN,PHI,...,-1.558217,120.127344,116.020505,4.106839,126.551076,124.565416,1.985660,120.409862,118.949371,1.460492
3,3,PHI 76ers,-6.0,U 224.5−110,−225,−115,−110,224.5,PHI,BKN,...,1.558217,116.020505,120.127344,-4.106839,124.565416,126.551076,-1.985660,118.949371,120.409862,-1.460492
4,4,ATL Hawks,-1.5,O 238−110,−125,−110,−110,238.0,ATL,OKC,...,0.545577,112.539423,112.834515,-0.295092,122.906163,117.349358,5.556806,116.622827,114.203091,2.419737


In [25]:
def get_net_score(team, matchup, cutoff):
    team_data = expected_minutes2[expected_minutes2['trad_team'] == team]
    team_data = team_data[team_data['minutes'] > 0]
    team_data = team_data.sort_values(by='PM_Player_Metric', ascending=False)
    team_data = team_data[:cutoff]
    # sum PM_Player_Metric
    team_score = team_data['PM_Player_Metric'].sum()

    matchup_data = expected_minutes2[expected_minutes2['trad_team'] == matchup]
    matchup_data = matchup_data[matchup_data['minutes'] > 0]
    matchup_data = matchup_data.sort_values(by='PM_Player_Metric', ascending=False)
    matchup_data = matchup_data[:cutoff]
    # sum PM_Player_Metric
    matchup_score = matchup_data['PM_Player_Metric'].sum()

    net_score = team_score - matchup_score
    return net_score

In [26]:
# test function
get_net_score('POR', 'IND', 5)

0.20833476021567776

In [27]:
today_df['net_score_top_4'] = today_df.apply(lambda x: get_net_score(x['trad_team'], x['matchup'], 4), axis=1)
today_df['net_score_top_5'] = today_df.apply(lambda x: get_net_score(x['trad_team'], x['matchup'], 5), axis=1)
today_df['net_score_top_6'] = today_df.apply(lambda x: get_net_score(x['trad_team'], x['matchup'], 6), axis=1)
today_df['net_score_top_7'] = today_df.apply(lambda x: get_net_score(x['trad_team'], x['matchup'], 7), axis=1)
today_df['net_score_top_8'] = today_df.apply(lambda x: get_net_score(x['trad_team'], x['matchup'], 8), axis=1)

# add predicted_winner
today_df['top_4_predicted_winner'] = np.where(today_df['net_score_top_4'] > 0, 1, 0)
today_df['top_5_predicted_winner'] = np.where(today_df['net_score_top_5'] > 0, 1, 0)
today_df['top_6_predicted_winner'] = np.where(today_df['net_score_top_6'] > 0, 1, 0)
today_df['top_7_predicted_winner'] = np.where(today_df['net_score_top_7'] > 0, 1, 0)
today_df['top_8_predicted_winner'] = np.where(today_df['net_score_top_8'] > 0, 1, 0)

today_df.head()

,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,net_score_top_4,net_score_top_5,net_score_top_6,net_score_top_7,net_score_top_8,top_4_predicted_winner,top_5_predicted_winner,top_6_predicted_winner,top_7_predicted_winner,top_8_predicted_winner
0,0,IND Pacers,5.5,O 234−110,+165,−115,−110,234.0,IND,ORL,...,-0.230433,-0.218763,-0.224639,-0.177959,-0.165322,0,0,0,0,0
1,1,ORL Magic,-5.5,U 234−110,−195,−105,−110,234.0,ORL,IND,...,0.230433,0.218763,0.224639,0.177959,0.165322,1,1,1,1,1
2,2,BKN Nets,6.0,O 224.5−110,+190,−105,−110,224.5,BKN,PHI,...,-1.114521,-1.411907,-1.684675,-2.732026,-3.650684,0,0,0,0,0
3,3,PHI 76ers,-6.0,U 224.5−110,−225,−115,−110,224.5,PHI,BKN,...,1.114521,1.411907,1.684675,2.732026,3.650684,1,1,1,1,1
4,4,ATL Hawks,-1.5,O 238−110,−125,−110,−110,238.0,ATL,OKC,...,0.565835,0.623060,0.652575,0.652603,0.651480,1,1,1,1,1


In [28]:
# check pre-processed data
ppd = pd.read_csv('data/team/aggregates/Pre-processed_data.csv')
ppd.head()

,Unnamed: 0,trad_season,t1_running_trad_pts,t1_running_trad_fgm,t1_running_trad_fga,t1_running_trad_fg%,t1_running_trad_3pm,t1_running_trad_3pa,t1_running_trad_3p%,t1_running_trad_ftm,...,Estimate_Points_Difference_Weighted,TEAM\nREST DAYS,OPENING ODDS,OPENING SPREAD,OPENING TOTAL,CLOSING\nODDS,CLOSING SPREAD,CLOSING TOTAL,MONEYLINE,home?
0,0,2022,113.096774,42.032258,87.000000,0.484000,10.516129,28.838710,0.358290,18.516129,...,-600.867949,3IN4,225.5o -15,5.5,225.5,225.5o -10,6.0,225.5,194.0,0
1,1,2022,113.096774,42.032258,87.000000,0.484000,10.516129,28.838710,0.358290,18.516129,...,-600.867949,3IN4,225.5o -15,5.5,225.5,225.5o -10,6.0,225.5,194.0,0
2,2,2022,110.764706,39.294118,87.441176,0.449294,11.411765,33.029412,0.342118,20.764706,...,-536.424138,3IN4,232.5o -15,8.5,232.5,233.5o -10,9.5,233.5,340.0,0
3,3,2022,110.764706,39.294118,87.441176,0.449294,11.411765,33.029412,0.342118,20.764706,...,-536.424138,3IN4,232.5o -15,8.5,232.5,233.5o -10,9.5,233.5,340.0,0
4,4,2022,107.575758,38.969697,84.757576,0.462273,12.272727,33.242424,0.368939,17.363636,...,-330.839448,1,216.5o -10,3.5,216.5,214o -10,3.5,214.0,142.0,0


In [29]:
# check columns in ppd that are not in today_df
ppd_cols = ppd.columns
today_df_cols = today_df.columns
ppd_cols.difference(today_df_cols)


Index(['CLOSING\nODDS', 'CLOSING SPREAD', 'CLOSING TOTAL', 'OPENING ODDS',
       'OPENING SPREAD', 'OPENING TOTAL', 'TEAM\nREST DAYS', 'Unnamed: 0',
       'home?', 'win?'],
      dtype='object')

In [30]:
today_df['CLOSING\nODDS'] = today_df['MONEYLINE']
today_df['OPENING ODDS'] = today_df['MONEYLINE']
today_df['CLOSING SPREAD'] = today_df['SPREAD']
today_df['OPENING SPREAD'] = today_df['SPREAD']
today_df['CLOSING TOTAL'] = today_df['TOTAL']
today_df['OPENING TOTAL'] = today_df['TOTAL']

In [31]:
# get just string columns from today_df
today_df_str = today_df.select_dtypes(include=['object'])
today_df_str.columns

Index(['TODAY', 'O/U', 'MONEYLINE', 'spread_odds', 'total_odds', 'trad_team',
       'matchup', 'AwayTeam', 'HomeTeam', 'trad_matchup', 'team_2',
       'trad_gamedate', 'home_or_away', 'CLOSING\nODDS', 'OPENING ODDS'],
      dtype='object')

In [32]:
today_df_str

,TODAY,O/U,MONEYLINE,spread_odds,total_odds,trad_team,matchup,AwayTeam,HomeTeam,trad_matchup,team_2,trad_gamedate,home_or_away,CLOSING\nODDS,OPENING ODDS
0,IND Pacers,O 234−110,+165,−115,−110,IND,ORL,IND,ORL,IND @ ORL,ORL,2023-01-25,away,+165,+165
1,ORL Magic,U 234−110,−195,−105,−110,ORL,IND,IND,ORL,IND @ ORL,IND,2023-01-25,home,−195,−195
2,BKN Nets,O 224.5−110,+190,−105,−110,BKN,PHI,BKN,PHI,BKN @ PHI,PHI,2023-01-25,away,+190,+190
3,PHI 76ers,U 224.5−110,−225,−115,−110,PHI,BKN,BKN,PHI,BKN @ PHI,BKN,2023-01-25,home,−225,−225
4,ATL Hawks,O 238−110,−125,−110,−110,ATL,OKC,ATL,OKC,ATL @ OKC,OKC,2023-01-25,away,−125,−125
5,OKC Thunder,U 238−110,+105,−110,−110,OKC,ATL,ATL,OKC,ATL @ OKC,ATL,2023-01-25,home,+105,+105
6,DEN Nuggets,O 229.5−110,+370,−110,−110,DEN,MIL,DEN,MIL,DEN @ MIL,MIL,2023-01-25,away,+370,+370
7,MIL Bucks,U 229.5−110,−460,−110,−110,MIL,DEN,DEN,MIL,DEN @ MIL,DEN,2023-01-25,home,−460,−460
8,WAS Wizards,O 232.5−110,−140,−115,−110,WAS,HOU,WAS,HOU,WAS @ HOU,HOU,2023-01-25,away,−140,−140
9,HOU Rockets,U 232.5−110,+120,−105,−110,HOU,WAS,WAS,HOU,WAS @ HOU,WAS,2023-01-25,home,+120,+120


In [33]:
make_float= ['MONEYLINE', 'spread_odds', 'total_odds', 'CLOSING\nODDS', 'OPENING ODDS']

for col in make_float:
    # change all to float after removing the + and changing the - to a negative
    today_df[col] = today_df[col].str.replace('+', '')
    today_df[col] = today_df[col].str.replace('−', '-')
    today_df[col] = today_df[col].astype(float)

today_df.head()



,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,top_5_predicted_winner,top_6_predicted_winner,top_7_predicted_winner,top_8_predicted_winner,CLOSING\nODDS,OPENING ODDS,CLOSING SPREAD,OPENING SPREAD,CLOSING TOTAL,OPENING TOTAL
0,0,IND Pacers,5.5,O 234−110,165.0,-115.0,-110.0,234.0,IND,ORL,...,0,0,0,0,165.0,165.0,5.5,5.5,234.0,234.0
1,1,ORL Magic,-5.5,U 234−110,-195.0,-105.0,-110.0,234.0,ORL,IND,...,1,1,1,1,-195.0,-195.0,-5.5,-5.5,234.0,234.0
2,2,BKN Nets,6.0,O 224.5−110,190.0,-105.0,-110.0,224.5,BKN,PHI,...,0,0,0,0,190.0,190.0,6.0,6.0,224.5,224.5
3,3,PHI 76ers,-6.0,U 224.5−110,-225.0,-115.0,-110.0,224.5,PHI,BKN,...,1,1,1,1,-225.0,-225.0,-6.0,-6.0,224.5,224.5
4,4,ATL Hawks,-1.5,O 238−110,-125.0,-110.0,-110.0,238.0,ATL,OKC,...,1,1,1,1,-125.0,-125.0,-1.5,-1.5,238.0,238.0


In [34]:
# drop infinity or NAN values
today_df = today_df.replace([np.inf, -np.inf], np.nan)
# check nan values
missing = today_df.isnull().sum()
missing[missing > 0]

team_salary                20
team_yoy_salary_change     20
team2_yoy_salary_change    20
dtype: int64

In [35]:
# replace nan with 0
today_df = today_df.fillna(0)

In [36]:
today_df.to_csv('data/team/aggregates/daily_updates/today_df_'+str(today)+'.csv', index=False)

In [37]:
ppd_cols = ppd.columns
today_df_cols = today_df.columns
ppd_cols.difference(today_df_cols)

Index(['TEAM\nREST DAYS', 'Unnamed: 0', 'home?', 'win?'], dtype='object')

In [38]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')

In [39]:
# load model
import pickle

with open('pickle_models/pipe_1.pkl', 'rb') as f:
    model = pickle.load(f)

In [40]:
data = pd.read_csv('data/team/aggregates/Pre-processed_data.csv')
data.head()


,Unnamed: 0,trad_season,t1_running_trad_pts,t1_running_trad_fgm,t1_running_trad_fga,t1_running_trad_fg%,t1_running_trad_3pm,t1_running_trad_3pa,t1_running_trad_3p%,t1_running_trad_ftm,...,Estimate_Points_Difference_Weighted,TEAM\nREST DAYS,OPENING ODDS,OPENING SPREAD,OPENING TOTAL,CLOSING\nODDS,CLOSING SPREAD,CLOSING TOTAL,MONEYLINE,home?
0,0,2022,113.096774,42.032258,87.000000,0.484000,10.516129,28.838710,0.358290,18.516129,...,-600.867949,3IN4,225.5o -15,5.5,225.5,225.5o -10,6.0,225.5,194.0,0
1,1,2022,113.096774,42.032258,87.000000,0.484000,10.516129,28.838710,0.358290,18.516129,...,-600.867949,3IN4,225.5o -15,5.5,225.5,225.5o -10,6.0,225.5,194.0,0
2,2,2022,110.764706,39.294118,87.441176,0.449294,11.411765,33.029412,0.342118,20.764706,...,-536.424138,3IN4,232.5o -15,8.5,232.5,233.5o -10,9.5,233.5,340.0,0
3,3,2022,110.764706,39.294118,87.441176,0.449294,11.411765,33.029412,0.342118,20.764706,...,-536.424138,3IN4,232.5o -15,8.5,232.5,233.5o -10,9.5,233.5,340.0,0
4,4,2022,107.575758,38.969697,84.757576,0.462273,12.272727,33.242424,0.368939,17.363636,...,-330.839448,1,216.5o -10,3.5,216.5,214o -10,3.5,214.0,142.0,0


In [41]:
data.columns.to_list()

['Unnamed: 0',
 'trad_season',
 't1_running_trad_pts',
 't1_running_trad_fgm',
 't1_running_trad_fga',
 't1_running_trad_fg%',
 't1_running_trad_3pm',
 't1_running_trad_3pa',
 't1_running_trad_3p%',
 't1_running_trad_ftm',
 't1_running_trad_fta',
 't1_running_trad_ft%',
 't1_running_trad_oreb',
 't1_running_trad_dreb',
 't1_running_trad_reb',
 't1_running_trad_ast',
 't1_running_trad_tov',
 't1_running_trad_stl',
 't1_running_trad_blk',
 't1_running_trad_pf',
 't1_running_trad_+/-',
 't1_running_adv_offrtg',
 't1_running_adv_defrtg',
 't1_running_adv_netrtg',
 't1_running_adv_ast%',
 't1_running_adv_ast/to',
 't1_running_adv_astratio',
 't1_running_adv_oreb%',
 't1_running_adv_dreb%',
 't1_running_adv_reb%',
 't1_running_adv_tov%',
 't1_running_adv_efg%',
 't1_running_adv_ts%',
 't1_running_adv_pace',
 't1_running_adv_pie',
 't1_running_four_efg%',
 't1_running_four_ftarate',
 't1_running_four_tov%',
 't1_running_four_oreb%',
 't1_running_four_oppefg%',
 't1_running_four_oppfta\xa0rate

In [42]:
test_cols = ['trad_season',
 't1_running_trad_pts',
 't1_running_trad_fgm',
 't1_running_trad_fga',
 't1_running_trad_fg%',
 't1_running_trad_3pm',
 't1_running_trad_3pa',
 't1_running_trad_3p%',
 't1_running_trad_ftm',
 't1_running_trad_fta',
 't1_running_trad_ft%',
 't1_running_trad_oreb',
 't1_running_trad_dreb',
 't1_running_trad_reb',
 't1_running_trad_ast',
 't1_running_trad_tov',
 't1_running_trad_stl',
 't1_running_trad_blk',
 't1_running_trad_pf',
 't1_running_trad_+/-',
 't1_running_adv_offrtg',
 't1_running_adv_defrtg',
 't1_running_adv_netrtg',
 't1_running_adv_ast%',
 't1_running_adv_ast/to',
 't1_running_adv_astratio',
 't1_running_adv_oreb%',
 't1_running_adv_dreb%',
 't1_running_adv_reb%',
 't1_running_adv_tov%',
 't1_running_adv_efg%',
 't1_running_adv_ts%',
 't1_running_adv_pace',
 't1_running_adv_pie',
 't1_running_four_efg%',
 't1_running_four_ftarate',
 't1_running_four_tov%',
 't1_running_four_oreb%',
 't1_running_four_oppefg%',
 't1_running_four_oppfta\xa0rate',
 't1_running_four_opptov%',
 't1_running_four_opporeb%',
 't1_running_misc_ptsoff\xa0to',
 't1_running_misc_2ndpts',
 't1_running_misc_fbps',
 't1_running_misc_pitp',
 't1_running_misc_opp\xa0ptsoff\xa0to',
 't1_running_misc_opp2nd\xa0pts',
 't1_running_misc_oppfbps',
 't1_running_misc_opppitp',
 't1_running_score_%fga2pt',
 't1_running_score_%fga3pt',
 't1_running_score_%pts2pt',
 't1_running_score_%pts2pt\xa0mr',
 't1_running_score_%pts3pt',
 't1_running_score_%ptsfbps',
 't1_running_score_%ptsft',
 't2_running_trad_pts',
 't2_running_trad_fgm',
 't2_running_trad_fga',
 't2_running_trad_fg%',
 't2_running_trad_3pm',
 't2_running_trad_3pa',
 't2_running_trad_3p%',
 't2_running_trad_ftm',
 't2_running_trad_fta',
 't2_running_trad_ft%',
 't2_running_trad_oreb',
 't2_running_trad_dreb',
 't2_running_trad_reb',
 't2_running_trad_ast',
 't2_running_trad_tov',
 't2_running_trad_stl',
 't2_running_trad_blk',
 't2_running_trad_pf',
 't2_running_trad_+/-',
 't2_running_adv_offrtg',
 't2_running_adv_defrtg',
 't2_running_adv_netrtg',
 't2_running_adv_ast%',
 't2_running_adv_ast/to',
 't2_running_adv_astratio',
 't2_running_adv_oreb%',
 't2_running_adv_dreb%',
 't2_running_adv_reb%',
 't2_running_adv_tov%',
 't2_running_adv_efg%',
 't2_running_adv_ts%',
 't2_running_adv_pace',
 't2_running_adv_pie',
 't2_running_four_efg%',
 't2_running_four_ftarate',
 't2_running_four_tov%',
 't2_running_four_oreb%',
 't2_running_four_oppefg%',
 't2_running_four_oppfta\xa0rate',
 't2_running_four_opptov%',
 't2_running_four_opporeb%',
 't2_running_misc_ptsoff\xa0to',
 't2_running_misc_2ndpts',
 't2_running_misc_fbps',
 't2_running_misc_pitp',
 't2_running_misc_opp\xa0ptsoff\xa0to',
 't2_running_misc_opp2nd\xa0pts',
 't2_running_misc_oppfbps',
 't2_running_misc_opppitp',
 't2_running_score_%fga2pt',
 't2_running_score_%fga3pt',
 't2_running_score_%pts2pt',
 't2_running_score_%pts2pt\xa0mr',
 't2_running_score_%pts3pt',
 't2_running_score_%ptsfbps',
 't2_running_score_%ptsft',
 'running_t1-t2_trad_pts',
 'running_t1-t2_trad_fgm',
 'running_t1-t2_trad_fga',
 'running_t1-t2_trad_fg%',
 'running_t1-t2_trad_3pm',
 'running_t1-t2_trad_3pa',
 'running_t1-t2_trad_3p%',
 'running_t1-t2_trad_ftm',
 'running_t1-t2_trad_fta',
 'running_t1-t2_trad_ft%',
 'running_t1-t2_trad_oreb',
 'running_t1-t2_trad_dreb',
 'running_t1-t2_trad_reb',
 'running_t1-t2_trad_ast',
 'running_t1-t2_trad_tov',
 'running_t1-t2_trad_stl',
 'running_t1-t2_trad_blk',
 'running_t1-t2_trad_pf',
 'running_t1-t2_trad_+/-',
 'running_t1-t2_adv_offrtg',
 'running_t1-t2_adv_defrtg',
 'running_t1-t2_adv_netrtg',
 'running_t1-t2_adv_ast%',
 'running_t1-t2_adv_ast/to',
 'running_t1-t2_adv_astratio',
 'running_t1-t2_adv_oreb%',
 'running_t1-t2_adv_dreb%',
 'running_t1-t2_adv_reb%',
 'running_t1-t2_adv_tov%',
 'running_t1-t2_adv_efg%',
 'running_t1-t2_adv_ts%',
 'running_t1-t2_adv_pace',
 'running_t1-t2_adv_pie',
 'running_t1-t2_four_efg%',
 'running_t1-t2_four_ftarate',
 'running_t1-t2_four_tov%',
 'running_t1-t2_four_oreb%',
 'running_t1-t2_four_oppefg%',
 'running_t1-t2_four_oppfta\xa0rate',
 'running_t1-t2_four_opptov%',
 'running_t1-t2_four_opporeb%',
 'running_t1-t2_misc_ptsoff\xa0to',
 'running_t1-t2_misc_2ndpts',
 'running_t1-t2_misc_fbps',
 'running_t1-t2_misc_pitp',
 'running_t1-t2_misc_opp\xa0ptsoff\xa0to',
 'running_t1-t2_misc_opp2nd\xa0pts',
 'running_t1-t2_misc_oppfbps',
 'running_t1-t2_misc_opppitp',
 'running_t1-t2_score_%fga2pt',
 'running_t1-t2_score_%fga3pt',
 'running_t1-t2_score_%pts2pt',
 'running_t1-t2_score_%pts2pt\xa0mr',
 'running_t1-t2_score_%pts3pt',
 'running_t1-t2_score_%ptsfbps',
 'running_t1-t2_score_%ptsft',
 't1_league_delta_trad_pts',
 't1_league_delta_trad_fgm',
 't1_league_delta_trad_fga',
 't1_league_delta_trad_fg%',
 't1_league_delta_trad_3pm',
 't1_league_delta_trad_3pa',
 't1_league_delta_trad_3p%',
 't1_league_delta_trad_ftm',
 't1_league_delta_trad_fta',
 't1_league_delta_trad_ft%',
 't1_league_delta_trad_oreb',
 't1_league_delta_trad_dreb',
 't1_league_delta_trad_reb',
 't1_league_delta_trad_ast',
 't1_league_delta_trad_tov',
 't1_league_delta_trad_stl',
 't1_league_delta_trad_blk',
 't1_league_delta_trad_pf',
 't1_league_delta_trad_+/-',
 't1_league_delta_adv_offrtg',
 't1_league_delta_adv_defrtg',
 't1_league_delta_adv_netrtg',
 't1_league_delta_adv_ast%',
 't1_league_delta_adv_ast/to',
 't1_league_delta_adv_astratio',
 't1_league_delta_adv_oreb%',
 't1_league_delta_adv_dreb%',
 't1_league_delta_adv_reb%',
 't1_league_delta_adv_tov%',
 't1_league_delta_adv_efg%',
 't1_league_delta_adv_ts%',
 't1_league_delta_adv_pace',
 't1_league_delta_adv_pie',
 't1_league_delta_four_efg%',
 't1_league_delta_four_ftarate',
 't1_league_delta_four_tov%',
 't1_league_delta_four_oreb%',
 't1_league_delta_four_oppefg%',
 't1_league_delta_four_oppfta\xa0rate',
 't1_league_delta_four_opptov%',
 't1_league_delta_four_opporeb%',
 't1_league_delta_misc_ptsoff\xa0to',
 't1_league_delta_misc_2ndpts',
 't1_league_delta_misc_fbps',
 't1_league_delta_misc_pitp',
 't1_league_delta_misc_opp\xa0ptsoff\xa0to',
 't1_league_delta_misc_opp2nd\xa0pts',
 't1_league_delta_misc_oppfbps',
 't1_league_delta_misc_opppitp',
 't1_league_delta_score_%fga2pt',
 't1_league_delta_score_%fga3pt',
 't1_league_delta_score_%pts2pt',
 't1_league_delta_score_%pts2pt\xa0mr',
 't1_league_delta_score_%pts3pt',
 't1_league_delta_score_%ptsfbps',
 't2_league_delta_trad_pts',
 't2_league_delta_trad_fgm',
 't2_league_delta_trad_fga',
 't2_league_delta_trad_fg%',
 't2_league_delta_trad_3pm',
 't2_league_delta_trad_3pa',
 't2_league_delta_trad_3p%',
 't2_league_delta_trad_ftm',
 't2_league_delta_trad_fta',
 't2_league_delta_trad_ft%',
 't2_league_delta_trad_oreb',
 't2_league_delta_trad_dreb',
 't2_league_delta_trad_reb',
 't2_league_delta_trad_ast',
 't2_league_delta_trad_tov',
 't2_league_delta_trad_stl',
 't2_league_delta_trad_blk',
 't2_league_delta_trad_pf',
 't2_league_delta_trad_+/-',
 't2_league_delta_adv_offrtg',
 't2_league_delta_adv_defrtg',
 't2_league_delta_adv_netrtg',
 't2_league_delta_adv_ast%',
 't2_league_delta_adv_ast/to',
 't2_league_delta_adv_astratio',
 't2_league_delta_adv_oreb%',
 't2_league_delta_adv_dreb%',
 't2_league_delta_adv_reb%',
 't2_league_delta_adv_tov%',
 't2_league_delta_adv_efg%',
 't2_league_delta_adv_ts%',
 't2_league_delta_adv_pace',
 't2_league_delta_adv_pie',
 't2_league_delta_four_efg%',
 't2_league_delta_four_ftarate',
 't2_league_delta_four_tov%',
 't2_league_delta_four_oreb%',
 't2_league_delta_four_oppefg%',
 't2_league_delta_four_oppfta\xa0rate',
 't2_league_delta_four_opptov%',
 't2_league_delta_four_opporeb%',
 't2_league_delta_misc_ptsoff\xa0to',
 't2_league_delta_misc_2ndpts',
 't2_league_delta_misc_fbps',
 't2_league_delta_misc_pitp',
 't2_league_delta_misc_opp\xa0ptsoff\xa0to',
 't2_league_delta_misc_opp2nd\xa0pts',
 't2_league_delta_misc_oppfbps',
 't2_league_delta_misc_opppitp',
 't2_league_delta_score_%fga2pt',
 't2_league_delta_score_%fga3pt',
 't2_league_delta_score_%pts2pt',
 't2_league_delta_score_%pts2pt\xa0mr',
 't2_league_delta_score_%pts3pt',
 't2_league_delta_score_%ptsfbps',
 't1_delta_minus_t2_delta_trad_pts',
 't1_delta_minus_t2_delta_trad_fgm',
 't1_delta_minus_t2_delta_trad_fga',
 't1_delta_minus_t2_delta_trad_fg%',
 't1_delta_minus_t2_delta_trad_3pm',
 't1_delta_minus_t2_delta_trad_3pa',
 't1_delta_minus_t2_delta_trad_3p%',
 't1_delta_minus_t2_delta_trad_ftm',
 't1_delta_minus_t2_delta_trad_fta',
 't1_delta_minus_t2_delta_trad_ft%',
 't1_delta_minus_t2_delta_trad_oreb',
 't1_delta_minus_t2_delta_trad_dreb',
 't1_delta_minus_t2_delta_trad_reb',
 't1_delta_minus_t2_delta_trad_ast',
 't1_delta_minus_t2_delta_trad_tov',
 't1_delta_minus_t2_delta_trad_stl',
 't1_delta_minus_t2_delta_trad_blk',
 't1_delta_minus_t2_delta_trad_pf',
 't1_delta_minus_t2_delta_trad_+/-',
 't1_delta_minus_t2_delta_adv_offrtg',
 't1_delta_minus_t2_delta_adv_defrtg',
 't1_delta_minus_t2_delta_adv_netrtg',
 't1_delta_minus_t2_delta_adv_ast%',
 't1_delta_minus_t2_delta_adv_ast/to',
 't1_delta_minus_t2_delta_adv_astratio',
 't1_delta_minus_t2_delta_adv_oreb%',
 't1_delta_minus_t2_delta_adv_dreb%',
 't1_delta_minus_t2_delta_adv_reb%',
 't1_delta_minus_t2_delta_adv_tov%',
 't1_delta_minus_t2_delta_adv_efg%',
 't1_delta_minus_t2_delta_adv_ts%',
 't1_delta_minus_t2_delta_adv_pace',
 't1_delta_minus_t2_delta_adv_pie',
 't1_delta_minus_t2_delta_four_efg%',
 't1_delta_minus_t2_delta_four_ftarate',
 't1_delta_minus_t2_delta_four_tov%',
 't1_delta_minus_t2_delta_four_oreb%',
 't1_delta_minus_t2_delta_four_oppefg%',
 't1_delta_minus_t2_delta_four_oppfta\xa0rate',
 't1_delta_minus_t2_delta_four_opptov%',
 't1_delta_minus_t2_delta_four_opporeb%',
 't1_delta_minus_t2_delta_misc_ptsoff\xa0to',
 't1_delta_minus_t2_delta_misc_2ndpts',
 't1_delta_minus_t2_delta_misc_fbps',
 't1_delta_minus_t2_delta_misc_pitp',
 't1_delta_minus_t2_delta_misc_opp\xa0ptsoff\xa0to',
 't1_delta_minus_t2_delta_misc_opp2nd\xa0pts',
 't1_delta_minus_t2_delta_misc_oppfbps',
 't1_delta_minus_t2_delta_misc_opppitp',
 't1_delta_minus_t2_delta_score_%fga2pt',
 't1_delta_minus_t2_delta_score_%fga3pt',
 't1_delta_minus_t2_delta_score_%pts2pt',
 't1_delta_minus_t2_delta_score_%pts2pt\xa0mr',
 't1_delta_minus_t2_delta_score_%pts3pt',
 't1_delta_minus_t2_delta_score_%ptsfbps',
 'team_wins_previous_season',
 'team_win%_previous_season',
 'team_2_wins_previous_season',
 'team2_win%_previous_season',
 'team1_prevwins_minus_team2_prevwins',
 'home_or_away',
 'prev_season_champ_winner',
 'team2_prev_season_champ_winner',
 'team_salary',
 'team_yoy_salary_change',
 'team2_yoy_salary_change',
 'rest_days',
 'team2_rest_days',
 'net_score_top_4',
 'net_score_top_5',
 'net_score_top_6',
 'net_score_top_7',
 'net_score_top_8',
 'top_4_predicted_winner',
 'top_5_predicted_winner',
 'top_6_predicted_winner',
 'top_7_predicted_winner',
 'top_8_predicted_winner',
 'tm1_80_game_avg_offrtg',
 'tm1_80_game_std_offrtg',
 'tm1_80_game_avg_defrtg',
 'tm1_80_game_std_defrtg',
 'tm1_80_game_avg_pace',
 'tm1_80_game_std_pace',
 'tm2_80_game_avg_offrtg',
 'tm2_80_game_std_offrtg',
 'tm2_80_game_avg_defrtg',
 'tm2_80_game_std_defrtg',
 'tm2_80_game_avg_pace',
 'tm2_80_game_std_pace',
 'tm1_40_gm_avg_offrtg',
 'tm1_40_gm_std_offrtg',
 'tm1_40_gm_avg_defrtg',
 'tm1_40_gm_std_defrtg',
 'tm1_40_gm_avg_pace',
 'tm1_40_gm_std_pace',
 'tm2_40_gm_avg_offrtg',
 'tm2_40_gm_std_offrtg',
 'tm2_40_gm_avg_defrtg',
 'tm2_40_gm_std_defrtg',
 'tm2_40_gm_avg_pace',
 'tm2_40_gm_std_pace',
 'tm1_20_gm_avg_offrtg',
 'tm1_20_gm_std_offrtg',
 'tm1_20_gm_avg_defrtg',
 'tm1_20_gm_std_defrtg',
 'tm1_20_gm_avg_pace',
 'tm1_20_gm_std_pace',
 'tm2_20_gm_avg_offrtg',
 'tm2_20_gm_std_offrtg',
 'tm2_20_gm_avg_defrtg',
 'tm2_20_gm_std_defrtg',
 'tm2_20_gm_avg_pace',
 'tm2_20_gm_std_pace',
 'tm1_10_gm_avg_offrtg',
 'tm1_10_gm_std_offrtg',
 'tm1_10_gm_avg_defrtg',
 'tm1_10_gm_std_defrtg',
 'tm1_10_gm_avg_pace',
 'tm1_10_gm_std_pace',
 'tm2_10_gm_avg_offrtg',
 'tm2_10_gm_std_offrtg',
 'tm2_10_gm_avg_defrtg',
 'tm2_10_gm_std_defrtg',
 'tm2_10_gm_avg_pace',
 'tm2_10_gm_std_pace',
 'nba_80_gm_avg_offrtg',
 'nba_80_gm_std_offrtg',
 'nba_80_gm_avg_defrtg',
 'nba_80_gm_std_defrtg',
 'nba_80_gm_avg_pace',
 'nba_80_gm_std_pace',
 'nba_40_gm_avg_offrtg',
 'nba_40_gm_std_offrtg',
 'nba_40_gm_avg_defrtg',
 'nba_40_gm_std_defrtg',
 'nba_40_gm_avg_pace',
 'nba_40_gm_std_pace',
 'nba_20_gm_avg_offrtg',
 'nba_20_gm_std_offrtg',
 'nba_20_gm_avg_defrtg',
 'nba_20_gm_std_defrtg',
 'nba_20_gm_avg_pace',
 'nba_20_gm_std_pace',
 'nba_10_gm_avg_offrtg',
 'nba_10_gm_std_offrtg',
 'nba_10_gm_avg_defrtg',
 'nba_10_gm_std_defrtg',
 'nba_10_gm_avg_pace',
 'nba_10_gm_std_pace',
 '80gm_tm1_minus_nba_avg_offrtg',
 '80gm_tm1_minus_nba_std_offrtg',
 '80gm_tm1_minus_nba_avg_defrtg',
 '80gm_tm1_minus_nba_std_defrtg',
 '80gm_tm1_minus_nba_avg_pace',
 '80gm_tm1_minus_nba_std_pace',
 '80gm_tm2_minus_nba_avg_offrtg',
 '80gm_tm2_minus_nba_std_offrtg',
 '80gm_tm2_minus_nba_avg_defrtg',
 '80gm_tm2_minus_nba_std_defrtg',
 '80gm_tm2_minus_nba_avg_pace',
 '80gm_tm2_minus_nba_std_pace',
 '40gm_tm1_minus_nba_avg_offrtg',
 '40gm_tm1_minus_nba_std_offrtg',
 '40gm_tm1_minus_nba_avg_defrtg',
 '40gm_tm1_minus_nba_std_defrtg',
 '40gm_tm1_minus_nba_avg_pace',
 '40gm_tm1_minus_nba_std_pace',
 '40gm_tm2_minus_nba_avg_offrtg',
 '40gm_tm2_minus_nba_std_offrtg',
 '40gm_tm2_minus_nba_avg_defrtg',
 '40gm_tm2_minus_nba_std_defrtg',
 '40gm_tm2_minus_nba_avg_pace',
 '40gm_tm2_minus_nba_std_pace',
 '20gm_tm1_minus_nba_avg_offrtg',
 '20gm_tm1_minus_nba_std_offrtg',
 '20gm_tm1_minus_nba_avg_defrtg',
 '20gm_tm1_minus_nba_std_defrtg',
 '20gm_tm1_minus_nba_avg_pace',
 '20gm_tm1_minus_nba_std_pace',
 '20gm_tm2_minus_nba_avg_offrtg',
 '20gm_tm2_minus_nba_std_offrtg',
 '20gm_tm2_minus_nba_avg_defrtg',
 '20gm_tm2_minus_nba_std_defrtg',
 '20gm_tm2_minus_nba_avg_pace',
 '20gm_tm2_minus_nba_std_pace',
 '10gm_tm1_minus_nba_avg_offrtg',
 '10gm_tm1_minus_nba_std_offrtg',
 '10gm_tm1_minus_nba_avg_defrtg',
 '10gm_tm1_minus_nba_std_defrtg',
 '10gm_tm1_minus_nba_avg_pace',
 '10gm_tm1_minus_nba_std_pace',
 '10gm_tm2_minus_nba_avg_offrtg',
 '10gm_tm2_minus_nba_std_offrtg',
 '10gm_tm2_minus_nba_avg_defrtg',
 '10gm_tm2_minus_nba_std_defrtg',
 '10gm_tm2_minus_nba_avg_pace',
 '10gm_tm2_minus_nba_std_pace',
 'Tm1_Points_Estimate_80gm',
 'Tm2_Points_Estimate_80gm',
 'Estimate_Points_Difference_80gm',
 'Tm1_Points_Estimate_40gm',
 'Tm2_Points_Estimate_40gm',
 'Estimate_Points_Difference_40gm',
 'Tm1_Points_Estimate_20gm',
 'Tm2_Points_Estimate_20gm',
 'Estimate_Points_Difference_20gm',
 'Tm1_Points_Estimate_Weighted',
 'Tm2_Points_Estimate_Weighted',
 'Estimate_Points_Difference_Weighted',
 'OPENING ODDS',
 'OPENING SPREAD',
 'OPENING TOTAL',
 'CLOSING\nODDS',
 'CLOSING SPREAD',
 'CLOSING TOTAL',
 'MONEYLINE']

In [43]:
today_df_2 = today_df[test_cols]
today_df_2.head(3)

,trad_season,t1_running_trad_pts,t1_running_trad_fgm,t1_running_trad_fga,t1_running_trad_fg%,t1_running_trad_3pm,t1_running_trad_3pa,t1_running_trad_3p%,t1_running_trad_ftm,t1_running_trad_fta,...,Tm1_Points_Estimate_Weighted,Tm2_Points_Estimate_Weighted,Estimate_Points_Difference_Weighted,OPENING ODDS,OPENING SPREAD,OPENING TOTAL,CLOSING\nODDS,CLOSING SPREAD,CLOSING TOTAL,MONEYLINE
0,2022,114.575758,40.878788,89.333333,0.459121,14.878788,40.212121,0.369242,17.939394,22.787879,...,113.196049,116.252836,-3.056787,165.0,5.5,234.0,165.0,5.5,234.0,165.0
1,2022,110.294118,39.735294,84.647059,0.470853,10.882353,31.235294,0.349353,19.941176,24.794118,...,116.252836,113.196049,3.056787,-195.0,-5.5,234.0,-195.0,-5.5,234.0,-195.0
2,2022,114.121212,42.606061,83.878788,0.508364,12.000000,31.212121,0.382030,16.909091,21.090909,...,120.409862,118.949371,1.460492,190.0,6.0,224.5,190.0,6.0,224.5,190.0


In [44]:
# convert OPENING ODDS to numeric... problem because it has plus and minus signs
today_df_2['OPENING ODDS'] = today_df_2['OPENING ODDS'].astype(str).str.strip()
today_df_2['OPENING ODDS'] = today_df_2['OPENING ODDS'].str.replace('+', '')
today_df_2['OPENING ODDS'] = today_df_2['OPENING ODDS'].str.replace('−', '-')
# to float
today_df_2['OPENING ODDS'] = today_df_2['OPENING ODDS'].astype(float)

today_df_2['CLOSING\nODDS'] = today_df_2['CLOSING\nODDS'].astype(str).str.strip()
today_df_2['CLOSING\nODDS'] = today_df_2['CLOSING\nODDS'].str.replace('+', '')
today_df_2['CLOSING\nODDS'] = today_df_2['CLOSING\nODDS'].str.replace('−', '-')
# to float
today_df_2['CLOSING\nODDS'] = today_df_2['CLOSING\nODDS'].astype(float)

today_df_2['MONEYLINE'] = today_df_2['MONEYLINE'].astype(str).str.strip()
today_df_2['MONEYLINE'] = today_df_2['MONEYLINE'].str.replace('+', '')
today_df_2['MONEYLINE'] = today_df_2['MONEYLINE'].str.replace('−', '-')
# to float
today_df_2['MONEYLINE'] = today_df_2['MONEYLINE'].astype(float)

today_df_2.head()

,trad_season,t1_running_trad_pts,t1_running_trad_fgm,t1_running_trad_fga,t1_running_trad_fg%,t1_running_trad_3pm,t1_running_trad_3pa,t1_running_trad_3p%,t1_running_trad_ftm,t1_running_trad_fta,...,Tm1_Points_Estimate_Weighted,Tm2_Points_Estimate_Weighted,Estimate_Points_Difference_Weighted,OPENING ODDS,OPENING SPREAD,OPENING TOTAL,CLOSING\nODDS,CLOSING SPREAD,CLOSING TOTAL,MONEYLINE
0,2022,114.575758,40.878788,89.333333,0.459121,14.878788,40.212121,0.369242,17.939394,22.787879,...,113.196049,116.252836,-3.056787,165.0,5.5,234.0,165.0,5.5,234.0,165.0
1,2022,110.294118,39.735294,84.647059,0.470853,10.882353,31.235294,0.349353,19.941176,24.794118,...,116.252836,113.196049,3.056787,-195.0,-5.5,234.0,-195.0,-5.5,234.0,-195.0
2,2022,114.121212,42.606061,83.878788,0.508364,12.000000,31.212121,0.382030,16.909091,21.090909,...,120.409862,118.949371,1.460492,190.0,6.0,224.5,190.0,6.0,224.5,190.0
3,2022,111.483871,39.741935,83.709677,0.476065,12.612903,32.935484,0.379355,19.387097,23.677419,...,118.949371,120.409862,-1.460492,-225.0,-6.0,224.5,-225.0,-6.0,224.5,-225.0
4,2022,114.575758,43.151515,92.333333,0.468667,10.333333,31.090909,0.333727,17.939394,21.696970,...,116.622827,114.203091,2.419737,-125.0,-1.5,238.0,-125.0,-1.5,238.0,-125.0


In [45]:
# check for non numeric dtypes only
today_df_2.select_dtypes(exclude=['int64', 'float64'])

,home_or_away,top_4_predicted_winner,top_5_predicted_winner,top_6_predicted_winner,top_7_predicted_winner,top_8_predicted_winner
0,away,0,0,0,0,0
1,home,1,1,1,1,1
2,away,0,0,0,0,0
3,home,1,1,1,1,1
4,away,1,1,1,1,1
5,home,0,0,0,0,0
6,away,0,0,0,0,0
7,home,1,1,1,1,1
8,away,1,1,1,1,1
9,home,0,0,0,0,0


In [46]:
today_df_2['home?']= np.where(today_df_2['home_or_away'] == 'home', 1, 0)

In [47]:
today_df_2.fillna(0, inplace=True)

In [48]:
today_df_2

,trad_season,t1_running_trad_pts,t1_running_trad_fgm,t1_running_trad_fga,t1_running_trad_fg%,t1_running_trad_3pm,t1_running_trad_3pa,t1_running_trad_3p%,t1_running_trad_ftm,t1_running_trad_fta,...,Tm2_Points_Estimate_Weighted,Estimate_Points_Difference_Weighted,OPENING ODDS,OPENING SPREAD,OPENING TOTAL,CLOSING\nODDS,CLOSING SPREAD,CLOSING TOTAL,MONEYLINE,home?
0,2022,114.575758,40.878788,89.333333,0.459121,14.878788,40.212121,0.369242,17.939394,22.787879,...,116.252836,-3.056787,165.0,5.5,234.0,165.0,5.5,234.0,165.0,0
1,2022,110.294118,39.735294,84.647059,0.470853,10.882353,31.235294,0.349353,19.941176,24.794118,...,113.196049,3.056787,-195.0,-5.5,234.0,-195.0,-5.5,234.0,-195.0,1
2,2022,114.121212,42.606061,83.878788,0.508364,12.000000,31.212121,0.382030,16.909091,21.090909,...,118.949371,1.460492,190.0,6.0,224.5,190.0,6.0,224.5,190.0,0
3,2022,111.483871,39.741935,83.709677,0.476065,12.612903,32.935484,0.379355,19.387097,23.677419,...,120.409862,-1.460492,-225.0,-6.0,224.5,-225.0,-6.0,224.5,-225.0,1
4,2022,114.575758,43.151515,92.333333,0.468667,10.333333,31.090909,0.333727,17.939394,21.696970,...,114.203091,2.419737,-125.0,-1.5,238.0,-125.0,-1.5,238.0,-125.0,0
5,2022,115.393939,42.757576,92.878788,0.462061,11.575758,33.666667,0.342121,18.303030,23.030303,...,116.622827,-2.419737,105.0,1.5,238.0,105.0,1.5,238.0,105.0,1
6,2022,115.580645,43.548387,86.129032,0.506677,11.580645,29.774194,0.383000,16.903226,22.967742,...,112.501795,4.207167,370.0,10.0,229.5,370.0,10.0,229.5,370.0,0
7,2022,112.468750,41.000000,88.906250,0.462750,13.031250,37.281250,0.353031,17.437500,23.562500,...,116.708961,-4.207167,-460.0,-10.0,229.5,-460.0,-10.0,229.5,-460.0,1
8,2022,111.176471,41.558824,87.029412,0.478971,10.970588,31.764706,0.344559,17.088235,22.529412,...,105.603567,4.186474,-140.0,-2.0,232.5,-140.0,-2.0,232.5,-140.0,0
9,2022,109.437500,39.125000,88.281250,0.443969,11.281250,34.218750,0.322219,19.906250,25.656250,...,109.790042,-4.186474,120.0,2.0,232.5,120.0,2.0,232.5,120.0,1


In [49]:
today_df

,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,top_5_predicted_winner,top_6_predicted_winner,top_7_predicted_winner,top_8_predicted_winner,CLOSING\nODDS,OPENING ODDS,CLOSING SPREAD,OPENING SPREAD,CLOSING TOTAL,OPENING TOTAL
0,0,IND Pacers,5.5,O 234−110,165.0,-115.0,-110.0,234.0,IND,ORL,...,0,0,0,0,165.0,165.0,5.5,5.5,234.0,234.0
1,1,ORL Magic,-5.5,U 234−110,-195.0,-105.0,-110.0,234.0,ORL,IND,...,1,1,1,1,-195.0,-195.0,-5.5,-5.5,234.0,234.0
2,2,BKN Nets,6.0,O 224.5−110,190.0,-105.0,-110.0,224.5,BKN,PHI,...,0,0,0,0,190.0,190.0,6.0,6.0,224.5,224.5
3,3,PHI 76ers,-6.0,U 224.5−110,-225.0,-115.0,-110.0,224.5,PHI,BKN,...,1,1,1,1,-225.0,-225.0,-6.0,-6.0,224.5,224.5
4,4,ATL Hawks,-1.5,O 238−110,-125.0,-110.0,-110.0,238.0,ATL,OKC,...,1,1,1,1,-125.0,-125.0,-1.5,-1.5,238.0,238.0
5,5,OKC Thunder,1.5,U 238−110,105.0,-110.0,-110.0,238.0,OKC,ATL,...,0,0,0,0,105.0,105.0,1.5,1.5,238.0,238.0
6,6,DEN Nuggets,10.0,O 229.5−110,370.0,-110.0,-110.0,229.5,DEN,MIL,...,0,0,0,0,370.0,370.0,10.0,10.0,229.5,229.5
7,7,MIL Bucks,-10.0,U 229.5−110,-460.0,-110.0,-110.0,229.5,MIL,DEN,...,1,1,1,1,-460.0,-460.0,-10.0,-10.0,229.5,229.5
8,8,WAS Wizards,-2.0,O 232.5−110,-140.0,-115.0,-110.0,232.5,WAS,HOU,...,1,1,1,1,-140.0,-140.0,-2.0,-2.0,232.5,232.5
9,9,HOU Rockets,2.0,U 232.5−110,120.0,-105.0,-110.0,232.5,HOU,WAS,...,0,0,0,0,120.0,120.0,2.0,2.0,232.5,232.5


In [50]:
#if "pk" in values, replace with 0
today_df_2['OPENING SPREAD'] = today_df_2['OPENING SPREAD'].replace('pk', 0)
today_df_2['CLOSING SPREAD'] = today_df_2['CLOSING SPREAD'].replace('pk', 0)

In [51]:
prediction =model.predict(today_df_2)
prediction

array([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
      dtype=int64)

In [52]:
# get the predicted probabilities
probs = model.predict_proba(today_df_2)
probs

array([[0.52, 0.48],
       [0.54, 0.46],
       [0.62, 0.38],
       [0.53, 0.47],
       [0.47, 0.53],
       [0.49, 0.51],
       [0.59, 0.41],
       [0.41, 0.59],
       [0.48, 0.52],
       [0.61, 0.39],
       [0.53, 0.47],
       [0.56, 0.44],
       [0.52, 0.48],
       [0.5 , 0.5 ],
       [0.49, 0.51],
       [0.43, 0.57],
       [0.59, 0.41],
       [0.51, 0.49],
       [0.67, 0.33],
       [0.42, 0.58]])

In [53]:
# add the probabilities to the dataframe
today_df['probs'] = probs[:,1]
today_df['prediction'] = prediction
today_df.head()

,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,top_7_predicted_winner,top_8_predicted_winner,CLOSING\nODDS,OPENING ODDS,CLOSING SPREAD,OPENING SPREAD,CLOSING TOTAL,OPENING TOTAL,probs,prediction
0,0,IND Pacers,5.5,O 234−110,165.0,-115.0,-110.0,234.0,IND,ORL,...,0,0,165.0,165.0,5.5,5.5,234.0,234.0,0.48,0
1,1,ORL Magic,-5.5,U 234−110,-195.0,-105.0,-110.0,234.0,ORL,IND,...,1,1,-195.0,-195.0,-5.5,-5.5,234.0,234.0,0.46,0
2,2,BKN Nets,6.0,O 224.5−110,190.0,-105.0,-110.0,224.5,BKN,PHI,...,0,0,190.0,190.0,6.0,6.0,224.5,224.5,0.38,0
3,3,PHI 76ers,-6.0,U 224.5−110,-225.0,-115.0,-110.0,224.5,PHI,BKN,...,1,1,-225.0,-225.0,-6.0,-6.0,224.5,224.5,0.47,0
4,4,ATL Hawks,-1.5,O 238−110,-125.0,-110.0,-110.0,238.0,ATL,OKC,...,1,1,-125.0,-125.0,-1.5,-1.5,238.0,238.0,0.53,1


In [54]:
today_df

,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,top_7_predicted_winner,top_8_predicted_winner,CLOSING\nODDS,OPENING ODDS,CLOSING SPREAD,OPENING SPREAD,CLOSING TOTAL,OPENING TOTAL,probs,prediction
0,0,IND Pacers,5.5,O 234−110,165.0,-115.0,-110.0,234.0,IND,ORL,...,0,0,165.0,165.0,5.5,5.5,234.0,234.0,0.48,0
1,1,ORL Magic,-5.5,U 234−110,-195.0,-105.0,-110.0,234.0,ORL,IND,...,1,1,-195.0,-195.0,-5.5,-5.5,234.0,234.0,0.46,0
2,2,BKN Nets,6.0,O 224.5−110,190.0,-105.0,-110.0,224.5,BKN,PHI,...,0,0,190.0,190.0,6.0,6.0,224.5,224.5,0.38,0
3,3,PHI 76ers,-6.0,U 224.5−110,-225.0,-115.0,-110.0,224.5,PHI,BKN,...,1,1,-225.0,-225.0,-6.0,-6.0,224.5,224.5,0.47,0
4,4,ATL Hawks,-1.5,O 238−110,-125.0,-110.0,-110.0,238.0,ATL,OKC,...,1,1,-125.0,-125.0,-1.5,-1.5,238.0,238.0,0.53,1
5,5,OKC Thunder,1.5,U 238−110,105.0,-110.0,-110.0,238.0,OKC,ATL,...,0,0,105.0,105.0,1.5,1.5,238.0,238.0,0.51,1
6,6,DEN Nuggets,10.0,O 229.5−110,370.0,-110.0,-110.0,229.5,DEN,MIL,...,0,0,370.0,370.0,10.0,10.0,229.5,229.5,0.41,0
7,7,MIL Bucks,-10.0,U 229.5−110,-460.0,-110.0,-110.0,229.5,MIL,DEN,...,1,1,-460.0,-460.0,-10.0,-10.0,229.5,229.5,0.59,1
8,8,WAS Wizards,-2.0,O 232.5−110,-140.0,-115.0,-110.0,232.5,WAS,HOU,...,1,1,-140.0,-140.0,-2.0,-2.0,232.5,232.5,0.52,1
9,9,HOU Rockets,2.0,U 232.5−110,120.0,-105.0,-110.0,232.5,HOU,WAS,...,0,0,120.0,120.0,2.0,2.0,232.5,232.5,0.39,0


In [55]:
show_cols = ['TODAY', 'SPREAD', 'TOTAL', 'MONEYLINE', 'probs', 'prediction']
today_show = today_df[show_cols]
today_show

,TODAY,SPREAD,TOTAL,MONEYLINE,probs,prediction
0,IND Pacers,5.5,234.0,165.0,0.48,0
1,ORL Magic,-5.5,234.0,-195.0,0.46,0
2,BKN Nets,6.0,224.5,190.0,0.38,0
3,PHI 76ers,-6.0,224.5,-225.0,0.47,0
4,ATL Hawks,-1.5,238.0,-125.0,0.53,1
5,OKC Thunder,1.5,238.0,105.0,0.51,1
6,DEN Nuggets,10.0,229.5,370.0,0.41,0
7,MIL Bucks,-10.0,229.5,-460.0,0.59,1
8,WAS Wizards,-2.0,232.5,-140.0,0.52,1
9,HOU Rockets,2.0,232.5,120.0,0.39,0


In [56]:
today_df.to_csv('data/team/aggregates/daily_updates/predictions_today_'+str(today)+'.csv', index=False)

In [57]:
# ADD EV CALCULATIONS
# EV is the expected value of a bet, defined as the probability of winning times the payout minus the probability of losing times the bet size
# EV = (probability of winning * payout) - (probability of losing * bet size)

today_df['EV'] = (today_df['probs'] * today_df['CLOSING\nODDS']) - ((1-today_df['probs']) * 100)

In [58]:
today_df

,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,top_8_predicted_winner,CLOSING\nODDS,OPENING ODDS,CLOSING SPREAD,OPENING SPREAD,CLOSING TOTAL,OPENING TOTAL,probs,prediction,EV
0,0,IND Pacers,5.5,O 234−110,165.0,-115.0,-110.0,234.0,IND,ORL,...,0,165.0,165.0,5.5,5.5,234.0,234.0,0.48,0,27.20
1,1,ORL Magic,-5.5,U 234−110,-195.0,-105.0,-110.0,234.0,ORL,IND,...,1,-195.0,-195.0,-5.5,-5.5,234.0,234.0,0.46,0,-143.70
2,2,BKN Nets,6.0,O 224.5−110,190.0,-105.0,-110.0,224.5,BKN,PHI,...,0,190.0,190.0,6.0,6.0,224.5,224.5,0.38,0,10.20
3,3,PHI 76ers,-6.0,U 224.5−110,-225.0,-115.0,-110.0,224.5,PHI,BKN,...,1,-225.0,-225.0,-6.0,-6.0,224.5,224.5,0.47,0,-158.75
4,4,ATL Hawks,-1.5,O 238−110,-125.0,-110.0,-110.0,238.0,ATL,OKC,...,1,-125.0,-125.0,-1.5,-1.5,238.0,238.0,0.53,1,-113.25
5,5,OKC Thunder,1.5,U 238−110,105.0,-110.0,-110.0,238.0,OKC,ATL,...,0,105.0,105.0,1.5,1.5,238.0,238.0,0.51,1,4.55
6,6,DEN Nuggets,10.0,O 229.5−110,370.0,-110.0,-110.0,229.5,DEN,MIL,...,0,370.0,370.0,10.0,10.0,229.5,229.5,0.41,0,92.70
7,7,MIL Bucks,-10.0,U 229.5−110,-460.0,-110.0,-110.0,229.5,MIL,DEN,...,1,-460.0,-460.0,-10.0,-10.0,229.5,229.5,0.59,1,-312.40
8,8,WAS Wizards,-2.0,O 232.5−110,-140.0,-115.0,-110.0,232.5,WAS,HOU,...,1,-140.0,-140.0,-2.0,-2.0,232.5,232.5,0.52,1,-120.80
9,9,HOU Rockets,2.0,U 232.5−110,120.0,-105.0,-110.0,232.5,HOU,WAS,...,0,120.0,120.0,2.0,2.0,232.5,232.5,0.39,0,-14.20


In [59]:
tot_rows = today_df.shape[0]
tot_games = tot_rows / 2

In [60]:
team_to_abv = {'ATL Hawks': 'ATL',
                'BKN Nets': 'BKN',
                'BOS Celtics': 'BOS',
                'CHA Hornets': 'CHA',
                'CHI Bulls': 'CHI',
                'CLE Cavaliers': 'CLE',
                'DAL Mavericks': 'DAL',
                'DEN Nuggets': 'DEN',
                'DET Pistons': 'DET',
                'GS Warriors': 'GSW',
                'HOU Rockets': 'HOU',
                'IND Pacers': 'IND',
                'LA Clippers': 'LAC',
                'LA Lakers': 'LAL',
                'MEM Grizzlies': 'MEM',
                'MIA Heat': 'MIA',
                'MIL Bucks': 'MIL',
                'MIN Timberwolves': 'MIN',
                'NO Pelicans': 'NOP',
                'NY Knicks': 'NYK',
                'OKC Thunder': 'OKC',
                'ORL Magic': 'ORL',
                'PHI 76ers': 'PHI',
                'PHO Suns': 'PHX',
                'POR Trail Blazers': 'POR',
                'SAC Kings': 'SAC',
                'SA Spurs': 'SAS',
                'TOR Raptors': 'TOR',
                'UTA Jazz': 'UTA',
                'WAS Wizards': 'WAS'}

In [61]:
def get_game(game_num):
        game_df = today_df.iloc[game_num*2:game_num*2+2]
        return game_df

In [62]:
def fix_prediction(df):
        total_probs = df['probs'].sum()
        if total_probs > 1:
            # if the total probability is greater than 1, then we need to adjust the probabilities
            # get the difference between the total and 1
            df['diff'] = 1 / total_probs
            # multiply the probabilities by the difference
            df['new_probs'] = round(df['probs'] * df['diff'],2)
            # get the new total
            df['EV_Fixed'] = round((df['new_probs'] * df['CLOSING\nODDS']) - ((1-df['new_probs']) * 100),2)
            df['team_abv'] = df['TODAY'].map(team_to_abv)
            keep_columns = ['TODAY', 'team_abv', 'SPREAD', 'TOTAL', 'MONEYLINE', 'probs', 'prediction', 'EV', 'new_probs', 'EV_Fixed']
            df = df[keep_columns]
            return df

        elif total_probs < 1:
            # if the total probability is less than 1, then we need to adjust the probabilities
            # get the difference between the total and 1
            df['diff'] = 1 / total_probs
            # multiply the probabilities by the difference
            df['new_probs'] = round(df['probs'] * df['diff'],2)
            # get the new total
            df['EV_Fixed'] = round((df['new_probs'] * df['CLOSING\nODDS']) - ((1-df['new_probs']) * 100),2)
            df['team_abv'] = df['TODAY'].map(team_to_abv)
            keep_columns = ['TODAY', 'team_abv', 'SPREAD', 'TOTAL', 'MONEYLINE', 'probs', 'prediction', 'EV', 'new_probs', 'EV_Fixed']
            df = df[keep_columns]
            return df

        else:
            df['new_probs'] = df['probs'].round(2)
            df['EV_Fixed'] = round((df['new_probs'] * df['CLOSING\nODDS']) - ((1-df['new_probs']) * 100),2)
            df['team_abv'] = df['TODAY'].map(team_to_abv)
            keep_columns = ['TODAY', 'team_abv', 'SPREAD', 'TOTAL', 'MONEYLINE', 'probs', 'prediction', 'EV', 'new_probs', 'EV_Fixed']
            df = df[keep_columns]
            return df


In [63]:
g1 = get_game(0)
g1

,index,TODAY,SPREAD,O/U,MONEYLINE,spread_odds,total_odds,TOTAL,trad_team,matchup,...,top_8_predicted_winner,CLOSING\nODDS,OPENING ODDS,CLOSING SPREAD,OPENING SPREAD,CLOSING TOTAL,OPENING TOTAL,probs,prediction,EV
0,0,IND Pacers,5.5,O 234−110,165.0,-115.0,-110.0,234.0,IND,ORL,...,0,165.0,165.0,5.5,5.5,234.0,234.0,0.48,0,27.2
1,1,ORL Magic,-5.5,U 234−110,-195.0,-105.0,-110.0,234.0,ORL,IND,...,1,-195.0,-195.0,-5.5,-5.5,234.0,234.0,0.46,0,-143.7


In [64]:
g1 = fix_prediction(g1)
g1

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
0,IND Pacers,IND,5.5,234.0,165.0,0.48,0,27.2,0.51,35.15
1,ORL Magic,ORL,-5.5,234.0,-195.0,0.46,0,-143.7,0.49,-146.55


In [65]:
g2 = get_game(1)
g2 = fix_prediction(g2)
g2

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
2,BKN Nets,BKN,6.0,224.5,190.0,0.38,0,10.20,0.45,30.50
3,PHI 76ers,PHI,-6.0,224.5,-225.0,0.47,0,-158.75,0.55,-168.75


In [66]:
g3 = get_game(2)
g3 = fix_prediction(g3)
g3

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
4,ATL Hawks,ATL,-1.5,238.0,-125.0,0.53,1,-113.25,0.51,-112.75
5,OKC Thunder,OKC,1.5,238.0,105.0,0.51,1,4.55,0.49,0.45


In [67]:
g4 = get_game(3)
g4 = fix_prediction(g4)
g4

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
6,DEN Nuggets,DEN,10.0,229.5,370.0,0.41,0,92.7,0.41,92.7
7,MIL Bucks,MIL,-10.0,229.5,-460.0,0.59,1,-312.4,0.59,-312.4


In [68]:
g5 = get_game(4)
g5 = fix_prediction(g5)
g5

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
8,WAS Wizards,WAS,-2.0,232.5,-140.0,0.52,1,-120.8,0.57,-122.8
9,HOU Rockets,HOU,2.0,232.5,120.0,0.39,0,-14.2,0.43,-5.4


In [69]:
g6 = get_game(5)
g6 = fix_prediction(g6)
g6

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
10,MIN Timberwolves,MIN,3.0,228.5,130.0,0.47,0,8.1,0.52,19.6
11,NO Pelicans,NOP,-3.0,228.5,-150.0,0.44,0,-122.0,0.48,-124.0


In [70]:
g7 = get_game(6)
g7 = fix_prediction(g7)
g7

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
12,UTA Jazz,UTA,3.5,239.5,140.0,0.48,0,15.2,0.49,17.60
13,POR Trail Blazers,POR,-3.5,239.5,-165.0,0.50,0,-132.5,0.51,-133.15


In [71]:
g8 = get_game(7)
g8 = fix_prediction(g8)
g8

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
14,TOR Raptors,TOR,3.0,238.0,130.0,0.51,1,17.3,0.47,8.1
15,SAC Kings,SAC,-3.0,238.0,-150.0,0.57,1,-128.5,0.53,-126.5


In [72]:
g9 = get_game(8)
g9 = fix_prediction(g9)
g9

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
16,MEM Grizzlies,MEM,3.5,245.5,140.0,0.41,0,-1.60,0.46,10.4
17,GS Warriors,GSW,-3.5,245.5,-165.0,0.49,0,-131.85,0.54,-135.1


In [73]:
g10 = get_game(9)
g10 = fix_prediction(g10)
g10

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
18,SA Spurs,SAS,7.0,244.0,230.0,0.33,0,8.9,0.36,18.8
19,LA Lakers,LAL,-7.0,244.0,-275.0,0.58,1,-201.5,0.64,-212.0


In [74]:
# append all the games together
games = [g1, g2, g3, g4, g5, g6, g7, g8, g9, g10]
games_df = pd.concat(games)
games_df

,TODAY,team_abv,SPREAD,TOTAL,MONEYLINE,probs,prediction,EV,new_probs,EV_Fixed
0,IND Pacers,IND,5.5,234.0,165.0,0.48,0,27.20,0.51,35.15
1,ORL Magic,ORL,-5.5,234.0,-195.0,0.46,0,-143.70,0.49,-146.55
2,BKN Nets,BKN,6.0,224.5,190.0,0.38,0,10.20,0.45,30.50
3,PHI 76ers,PHI,-6.0,224.5,-225.0,0.47,0,-158.75,0.55,-168.75
4,ATL Hawks,ATL,-1.5,238.0,-125.0,0.53,1,-113.25,0.51,-112.75
5,OKC Thunder,OKC,1.5,238.0,105.0,0.51,1,4.55,0.49,0.45
6,DEN Nuggets,DEN,10.0,229.5,370.0,0.41,0,92.70,0.41,92.70
7,MIL Bucks,MIL,-10.0,229.5,-460.0,0.59,1,-312.40,0.59,-312.40
8,WAS Wizards,WAS,-2.0,232.5,-140.0,0.52,1,-120.80,0.57,-122.80
9,HOU Rockets,HOU,2.0,232.5,120.0,0.39,0,-14.20,0.43,-5.40


In [75]:
# scrape usage from basketball-reference.com daily
filename = 'data/team/aggregates/daily_updates/advanced_player_stats'+str(today)+'.csv'
if os.path.exists(filename):
    pass
else:
    advanced_stats = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_advanced.html')
    advanced_stats = advanced_stats[0]
    advanced_stats.to_csv('data/team/aggregates/daily_updates/advanced_player_stats'+str(today)+'.csv', index=False)
    advanced_stats

#### Better Usg Metric Update

In [76]:
file = 'data/player/aggregates/current_usg_metrics_'+ today +'.csv'
if os.path.exists(file):
    pass
else:
    os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')
    %run better_usg_metric.py


#### Scrape Team Playtypes

In [77]:
playtype_file = 'data/player/aggregates/playtype_usg_metrics_'+ today +'.csv'
if os.path.exists(playtype_file):
    pass
else:
    os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\Apps\\Team_Play_Type_Project')

    %run team_playtype_scrape.py

Already scraped today
Job Done


### Twitter bot addition

In [78]:
api_key = os.environ.get('Twitter_API_Key')
api_secret = os.environ.get('Twitter_API_Secret_Key')
access_token = os.environ.get('Twitter_Access_Token')
access_token_secret = os.environ.get('Twitter_Access_token_Secret')
bearer_token = os.environ.get('Twitter_Bearer_Token')

client = tweepy.Client(bearer_token, api_key, api_secret, access_token, access_token_secret)

auth = tweepy.OAuthHandler(api_key, api_secret, access_token, access_token_secret)
api = tweepy.API(auth)

In [79]:
team_name_dict= [{'team': 'Atlanta', 'abbreviation': 'ATL'},
                 {'team': 'Boston', 'abbreviation': 'BOS'},
                 {'team': 'Brooklyn', 'abbreviation': 'BKN'},
                 {'team': 'Charlotte', 'abbreviation': 'CHA'},
                 {'team': 'Chicago', 'abbreviation': 'CHI'},
                 {'team': 'Cleveland', 'abbreviation': 'CLE'},
                 {'team': 'Dallas', 'abbreviation': 'DAL'},
                 {'team': 'Denver', 'abbreviation': 'DEN'},
                 {'team': 'Detroit', 'abbreviation': 'DET'},
                 {'team': 'Golden State', 'abbreviation': 'GSW'},
                 {'team': 'Houston', 'abbreviation': 'HOU'},
                 {'team': 'Indiana', 'abbreviation': 'IND'},
                 {'team': 'LA Clippers', 'abbreviation': 'LAC'},
                 {'team': 'LA Lakers', 'abbreviation': 'LAL'},
                 {'team': 'Memphis', 'abbreviation': 'MEM'},
                 {'team': 'Miami', 'abbreviation': 'MIA'},
                 {'team': 'Milwaukee', 'abbreviation': 'MIL'},
                 {'team': 'Minnesota', 'abbreviation': 'MIN'},
                 {'team': 'New Orleans', 'abbreviation': 'NOP'},
                 {'team': 'New York', 'abbreviation': 'NYK'},
                 {'team': 'Oklahoma City', 'abbreviation': 'OKC'},
                 {'team': 'Orlando', 'abbreviation': 'ORL'},
                 {'team': 'Philadelphia', 'abbreviation': 'PHI'},
                 {'team': 'Phoenix', 'abbreviation': 'PHX'},
                 {'team': 'Portland', 'abbreviation': 'POR'},
                 {'team': 'Sacramento', 'abbreviation': 'SAC'},
                 {'team': 'San Antonio', 'abbreviation': 'SAS'},
                 {'team': 'Toronto', 'abbreviation': 'TOR'},
                 {'team': 'Utah', 'abbreviation': 'UTA'},
                 {'team': 'Washington', 'abbreviation': 'WAS'}]

In [80]:
# get current time
now = datetime.datetime.now()
current_time = now.strftime("%H:%M:%S")
# make current time normal, not military time
current_time = datetime.datetime.strptime(current_time, '%H:%M:%S').strftime('%I:%M %p')
print(current_time)

02:59 PM


In [81]:
today = datetime.date.today()
today = today.strftime('%Y-%m-%d')
odds_files = os.listdir('data/team/aggregates/daily_updates/')
odds_files = [file for file in odds_files if 'games_and_odds' in file]

if today in odds_files[0]:
    print('found today')
    big_odds_df = pd.read_csv('data/team/aggregates/daily_updates/' + odds_files[0])
    
else:

    # use oddsshark scrape
    url = 'https://www.oddsshark.com/nba/odds'

    driver = webdriver.Chrome()
    driver.get(url)

    # Get the html of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    data = soup.find('div', class_= 'odds--group__events-container')
    single_games = data.find_all(class_ = 'odds--group__event-container basketball')
    big_odds_df = pd.DataFrame()
    for game in single_games:
        # get teams
        teams = game.find(class_ = 'odds--group__event-participants').text.strip()
        teams = teams.replace('vs', '')
        teams = teams.replace(' ET', '')
        teams = teams.replace('  ', ' ')
        # get team 1
        # find index of first number
        index = 0
        for char in teams:
            if char.isdigit():
                break
            index += 1
        team1 = teams[:index].strip()
        # get time... from index of first number through "pm"
        pm = teams.find('pm')
        time = teams[index:pm]
        # add 12 hours to time if pm
        # get team 2
        team2 = teams[pm+2:].strip()
        # get odds
        spreads = game.find_all('div', 'odds-spread')

        odds_df = pd.DataFrame()

        n = 1
        for spread in spreads:
            spread = spread.text.replace('- ', '').replace(' - ', '').strip()[:-2][:-4]
            # opening spread
            if n == 1:
                opening_spread_t1 = spread
                n += 1
            elif n == 2:
                opening_spread_t2 = spread
                n += 1
            elif n == 3:
                current_spread_t1 = spread
                n += 1
            elif n == 4:
                current_spread_t2 = spread
                n += 1
        
        moneylines = game.find_all('div', 'odds-moneyline above-tablet-only')
        n = 1
        for moneyline in moneylines:
            moneyline = moneyline.text.replace('- ', '').replace(' - ', '').strip()
            # opening moneyline
            if n == 1:
                opening_moneyline_t1 = moneyline
                n += 1
            elif n == 2:
                opening_moneyline_t2 = moneyline
                n += 1
            elif n == 3:
                current_moneyline_t1 = moneyline
                n += 1
            elif n == 4:
                current_moneyline_t2 = moneyline
                n += 1

                # add to dataframe
                odds_df = odds_df.append({'team1': team1, 'team2': team2, 'time': time, 'opening_spread_t1': opening_spread_t1, 'opening_spread_t2': opening_spread_t2, 'current_spread_t1': current_spread_t1, 'current_spread_t2': current_spread_t2, 'opening_moneyline_t1': opening_moneyline_t1, 'opening_moneyline_t2': opening_moneyline_t2, 'current_moneyline_t1': current_moneyline_t1, 'current_moneyline_t2': current_moneyline_t2}, ignore_index=True)

        big_odds_df = big_odds_df.append(odds_df, ignore_index=True)

    # close driver
    driver.close()

    today = datetime.date.today()
    today = today.strftime('%Y-%m-%d')
    time_now = datetime.datetime.now().strftime('%H-%M')
    # add 12 hours to time
    big_odds_df['time'] = big_odds_df['time'] + ' PM'
    # add 12 hours
    big_odds_df['time'] = pd.to_datetime(big_odds_df['time'], format='%I:%M %p').dt.time
    # subtract 3 hours
    big_odds_df['time'] = big_odds_df['time'].apply(lambda x: x.replace(hour=x.hour-3))
    big_odds_df.to_csv('data/team/aggregates/daily_updates/games_and_odds_' + str(today) +'_'+ str(time_now) + '.csv')


big_odds_df


,team1,team2,time,opening_spread_t1,opening_spread_t2,current_spread_t1,current_spread_t2,opening_moneyline_t1,opening_moneyline_t2,current_moneyline_t1,current_moneyline_t2
0,Indiana,Orlando,16:00:00,+4,-4,+5,-5,+135,-185,+165,-190
1,Brooklyn,Philadelphia,16:30:00,+6.5,-6.5,+6,-6,+200,-294,+190,-220
2,Atlanta,Oklahoma City,17:00:00,-1,+1,-1.5,+1.5,-120,-111,-120,+100
3,Minnesota,New Orleans,17:00:00,+1.5,-1.5,+3,-3,-105,-125,+126,-146
4,Denver,Milwaukee,17:00:00,+6.5,-6.5,+10,-10,+210,-256,+375,-465
5,Washington,Houston,17:00:00,-2.5,+2.5,-2,+2,-147,+124,-133,+113
6,Memphis,Golden State,19:00:00,+2.5,-2.5,+3.5,-3.5,+120,-141,+140,-160
7,Toronto,Sacramento,19:00:00,+4.5,-4.5,+3,-3,+150,-213,+134,-154
8,Utah,Portland,19:00:00,+3.5,-3.5,+3.5,-3.5,+130,-182,+140,-160


In [82]:
now = datetime.datetime.now()
in_30 = now + datetime.timedelta(minutes=30)

print(f'time now: {now}, in 30: {in_30}')

time now: 2023-01-25 14:59:23.501946, in 30: 2023-01-25 15:29:23.501946


In [83]:
in_30_time = pd.to_datetime(in_30).time()
now_time = pd.to_datetime(now).time()
print(f'time now: {now_time}, in 30: {in_30_time}')

time now: 14:59:23.501946, in 30: 15:29:23.501946


In [84]:
big_odds_df['time']

0    16:00:00
1    16:30:00
2    17:00:00
3    17:00:00
4    17:00:00
5    17:00:00
6    19:00:00
7    19:00:00
8    19:00:00
Name: time, dtype: object

In [85]:
games_soon = big_odds_df[big_odds_df['time'] > now_time ]
games_soon = games_soon[games_soon['time'] < in_30_time]
games_soon

,team1,team2,time,opening_spread_t1,opening_spread_t2,current_spread_t1,current_spread_t2,opening_moneyline_t1,opening_moneyline_t2,current_moneyline_t1,current_moneyline_t2


In [86]:
# check confirmed lineups

if len(games_soon) > 0:

    driver = webdriver.Chrome()
    url = 'https://www.rotowire.com/basketball/nba-lineups.php'


    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # find div class = 'lineups'. This is the div that contains all the lineups.
    # The lineups are split by 'lineup is-nba' class

    tot_lineup_df = pd.DataFrame()

    lineups = soup.find_all('div', class_='lineup__box')
    # get rid of last 4 lineups
    lineups = lineups[:-4]

    for lineup in lineups:
        # wait for page to load

        away_team = lineup.find('ul', class_= 'lineup__list is-visit').text.strip().replace('\n', ' ')
        home_team = lineup.find('ul', class_= 'lineup__list is-home').text.strip().replace('\n', ' ')

            # split into three lists by 'Expected Lineup'
        away_team_name = lineup.find('a', class_='lineup__team is-visit').text.strip()
        home_team_name = lineup.find('a', class_='lineup__team is-home').text.strip()

        away_team_pg = away_team[away_team.find('PG')+3:away_team.find('SG')-1]
        away_team_sg = away_team[away_team.find('SG')+3:away_team.find('SF')-1]
        away_team_sf = away_team[away_team.find('SF')+3:away_team.find('PF')-1]
        away_team_pf = away_team[away_team.find('PF')+3:away_team.find(' C ')]
        away_team_c = away_team[away_team.find(' C ')+2:away_team.find('Pro')-1]

        home_team_pg = home_team[home_team.find('PG')+3:home_team.find('SG')-1]
        home_team_sg = home_team[home_team.find('SG')+3:home_team.find('SF')-1]
        home_team_sf = home_team[home_team.find('SF')+3:home_team.find('PF')-1]
        home_team_pf = home_team[home_team.find('PF')+3:home_team.find(' C ')]
        home_team_c = home_team[home_team.find(' C ')+2:home_team.find('Pro')-1]

        away_status = away_team[0:away_team.find('PG')-2]
        home_status = home_team[0:home_team.find('PG')-2]

        # create dataframe
        lineup_df = pd.DataFrame({'Away_Team': away_team_name, 'Home_Team': home_team_name, 'Away_PG': away_team_pg, 'Away_SG': away_team_sg, 'Away_SF': away_team_sf, 'Away_PF': away_team_pf, 'Away_C': away_team_c, 'Home_PG': home_team_pg, 'Home_SG': home_team_sg, 'Home_SF': home_team_sf, 'Home_PF': home_team_pf, 'Home_C': home_team_c, 'Away_Status': away_status, 'Home_Status': home_status}, index=[0])

        tot_lineup_df = pd.concat([tot_lineup_df, lineup_df], axis=0)

    tot_lineup_df = tot_lineup_df.reset_index(drop=True)

    today = datetime.datetime.now().strftime('%Y-%m-%d')

    tot_lineup_df.to_csv('data/team/aggregates/daily_updates/starting_lineups_{}.csv'.format(today), index=False)

    driver.close()

    # add abbrevs to games_soon
    games_soon['team1_abv'] = games_soon['team1'].map(lambda x: [d['abbreviation'] for d in team_name_dict if d['team'] == x][0])
    games_soon['team2_abv'] = games_soon['team2'].map(lambda x: [d['abbreviation'] for d in team_name_dict if d['team'] == x][0])

    # Merge, on Away_Team and Home_Team and team1_abv and team2_abv
    games_soon = pd.merge(games_soon, tot_lineup_df, left_on=['team1_abv', 'team2_abv'], right_on=['Away_Team', 'Home_Team'], how='left')
    
    # RUN TOTAL UPDATE
    if games_soon is not None:
        os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')
        %run 1_Daily_Update_Notebook.py
        today = datetime.datetime.today().strftime('%Y-%m-%d')

        predictions = pd.read_csv('data/team/aggregates/daily_updates/predictions_today_'+ str(today) +'.csv')

        # combine games_soon with predictions
        games_soon = pd.merge(games_soon, predictions, left_on=['team1_abv', 'team2_abv'], right_on=['AwayTeam', 'HomeTeam'], how='left')

        current_time = datetime.datetime.now().strftime('%H:%M:%S').replace(':', '-')
        games_soon.to_csv('data/team/aggregates/daily_updates/locked_in_preds_'+ str(today) + '_' + str(current_time) +'.csv', index=False)
        
        games_soon
    
    else:
        print('No games starting soon')

else:
    print('No games starting soon')

No games starting soon


In [87]:
big_odds_df['team1_abv'] = big_odds_df['team1'].map(lambda x: [d['abbreviation'] for d in team_name_dict if d['team'] == x][0])
big_odds_df['team2_abv'] = big_odds_df['team2'].map(lambda x: [d['abbreviation'] for d in team_name_dict if d['team'] == x][0])
big_odds_df

,team1,team2,time,opening_spread_t1,opening_spread_t2,current_spread_t1,current_spread_t2,opening_moneyline_t1,opening_moneyline_t2,current_moneyline_t1,current_moneyline_t2,team1_abv,team2_abv
0,Indiana,Orlando,16:00:00,+4,-4,+5,-5,+135,-185,+165,-190,IND,ORL
1,Brooklyn,Philadelphia,16:30:00,+6.5,-6.5,+6,-6,+200,-294,+190,-220,BKN,PHI
2,Atlanta,Oklahoma City,17:00:00,-1,+1,-1.5,+1.5,-120,-111,-120,+100,ATL,OKC
3,Minnesota,New Orleans,17:00:00,+1.5,-1.5,+3,-3,-105,-125,+126,-146,MIN,NOP
4,Denver,Milwaukee,17:00:00,+6.5,-6.5,+10,-10,+210,-256,+375,-465,DEN,MIL
5,Washington,Houston,17:00:00,-2.5,+2.5,-2,+2,-147,+124,-133,+113,WAS,HOU
6,Memphis,Golden State,19:00:00,+2.5,-2.5,+3.5,-3.5,+120,-141,+140,-160,MEM,GSW
7,Toronto,Sacramento,19:00:00,+4.5,-4.5,+3,-3,+150,-213,+134,-154,TOR,SAC
8,Utah,Portland,19:00:00,+3.5,-3.5,+3.5,-3.5,+130,-182,+140,-160,UTA,POR
